<a href="https://colab.research.google.com/github/BernadetteCB/UndergraduateProject/blob/main/UndergraduateProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# layoutgen()

In [ ]:
def layoutgen():
  # Produce a 10x10 grid of zero entries, with 'T' randomly chosen coordinates that each equal 1, where values 0 and 1 indicate the absence and presence of a turbine in the representative cell, respectively:
  ab=[] # Initialise an empty list in which the randomly-generated coordinates are stored
  T=rng.randint(30,60) # randomly generates the number of turbines
  while len(ab)<T:
    A=rng.randint(0,10) # randomly generates the x-coordinate
    B=rng.randint(0,10) # randomly genereates the y-coordinate
    AB=[A,B] # assigns the coordinate pair to variable AB
    # if it isn't, add it to list 'ab':
    if AB not in ab:
      ab.append(AB)
      continue
    # if it is, randomly generate a different coordinate pair:
    else:
      continue
  g=np.zeros(shape=(10,10),dtype=np.float64) # creates a 10x10 array of zeros
  # Assign each coordinate pair in list 'ab' to an entry in array 'G' equal to 1:
  for i in range (0,len(ab)):
    s,t=ab[i]
    g[s,t]=1

  return g

# output(g)

In [ ]:
def output(g):
  g0=None # initialise while loop condition
  # Define variables used to calculate wake region R in their original units:
  z=60 # height of wind turbines (meters)
  Ct=0.88 # thrust coefficient
  z0=0.3 # roughness of turain (meters)
  k=0.5/math.log(z/z0)
  r0=20 # rotor radius (meters)
  a=0.3267949192 # axial induction factor
  r=r0*math.sqrt((1-a)/(1-2*a))
  # Run until all powergrids are calculated:
  while not np.array_equal(g,g0):
    g0=g.copy()
    p=np.argwhere(g) # obtain coordinates of turbines
    points=np.argwhere(g.flatten()) # obtain 1-D coordinates of turbines
    G1=np.zeros(shape=(10,10)) # initialise storage for turbines affected by wake
    grids=[g] # store layoutgen() grid as first entry of grids
    innerloop=0 # initialise inner while-loop count
    eucdist=eucdistALL[np.ix_(points.flatten(),points.flatten())] # extarct pairwise distances between all turbines in g
    # Run until the power output of all turbines is calculated:
    while len(p)>0:
      p=p if innerloop==0 else np.argwhere(G1)
      affturb=[] # initialise storage for turbines affected by wake
      G1=np.zeros((10,10)) # clear storage for turbines affected by wake
      for i in range(len(p)):
        for j in range(len(p)):
          if p[i][0]<p[j][0] and p[j][0]-p[i][0]<5:
            x=(p[j][0]-p[i][0])*20 # difference in y-coordinates (min dist. must be 20m (rotor radius))
            R=(k*x)+r # wake radius in meters
            RCU=R/20 # convert wake region to coordinate-units (cell size is 20m)
            # Determine if the turbine is affected by wake (and add to affturb list):
            if eucdist[i,j]<RCU:
              affturb.append([p[j][0],p[j][1]])
      for n in affturb:
        G1[n[0],n[1]]=1 # assign values of 1 in G1 to the coordinates of affected turbines
      grids.append(G1.copy()) # add the grid of affected turbines to the grids list
      innerloop+=1 # increment while-loop count
    grids=np.asarray(grids,dtype=np.float64)
    # Calculate the individual turbine power outputs:
    powergrid=np.zeros(shape=(10,10),dtype=np.float64) # create storage for the overall power grid
    powergrid=grids[0]*630 # store powers as unaffected by wake
    for i in range(1,len(grids)):
      wakeeffect=grids[i]*630 # multiply the affected turbines by their power as unaffected by wake
      powergrid-=wakeeffect/3 # apply the wake effect
    powergrid=np.clip(powergrid,0,630) # discard turbine entries with negative power output
    # Create a 1-D array of an update 'g' of turbines with positive power output:
    flatg=np.zeros(shape=(100,1))
    gcoords=np.argwhere(powergrid.flatten()>0).flatten()
    for i in gcoords:
      flatg[i]=1
    g=np.reshape(flatg,(10,10)).astype(np.float64)
  totalturbs=len(np.argwhere(powergrid)) # calculate the number of turbines after wake effect is applied
  # Calculate total power output:
  totalpower=np.sum(powergrid) # sum all powergrids

  return g,powergrid,totalpower,totalturbs

# NSGA2 and SPEA2 optimisation

In [ ]:
!pip install -U pymoo
import pymoo
import os
from google.colab import files
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline
mpl.use('Agg')
from functools import partial
import numpy as np
from numpy import random as rng
import random
import math # (for output(g) function)
from scipy.spatial.distance import cdist
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.spea2 import SPEA2
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.callback import Callback
from pymoo.core.mutation import Mutation
from pymoo.core.crossover import Crossover
from pymoo.core.evaluator import Evaluator
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.optimize import minimize
from concurrent.futures import ProcessPoolExecutor

In [ ]:
# Precalculate and store all possible pariwise distances for each turbine:
pointsALL=np.array([(i,j) for i in range(10) for j in range(10)])
eucdistALL=cdist(pointsALL,pointsALL,metric='euclidean')

class CustomNSGA2(NSGA2):
  # Allow for custom classes in NSGA2, not inheritting from standard classes:
  def __init__(self,*args,**kwargs):
    super().__init__(*args,**kwargs)
    self.mutation=CustomMutation()
    self.crossover=CustomCrossover()

class CustomSPEA2(SPEA2):
  # Allow for custom classes in SPEA2, not inheritting from standard classes:
  def __init__(self,*args,**kwargs):
    super().__init__(*args,**kwargs)
    self.mutation=CustomMutation()
    self.crossover=CustomCrossover()

class CustomSampling(FloatRandomSampling):
  def _do(self,problem,n_samples,**kwargs):
    X = np.full((n_samples, problem.n_var),None,dtype=problem.xl.dtype)
    for i in range(n_samples):
      X[i]=np.array(layoutgen().flatten()) # produce the sampling array
    return X

class CustomMutation(Mutation):
  def __init__(self,prob=1.0,eta=5):
    super().__init__()

  def _do(self, problem, X, **kwargs):
    # Use partial to pass extra args to the function:
    indivmutF=partial(indivmut,algorithm=problem.algorithm)
    # Use executor to parallel-process each solution:
    with ProcessPoolExecutor() as executor:
      results=list(executor.map(indivmutF, X))
    # Update original population with mutated solutions:
    for i, result in enumerate(results):
      X[i]=result
    return X

def indivmut(x,algorithm=None):
    g=x.reshape((10, 10))
    g,powergrid,totalpower,totalturbs=output(g)
    AFFturb=np.ravel(np.argwhere(powergrid==1))  # turbines in wake
    poschoices=np.ravel(np.argwhere(powergrid==0))  # available positions
    if totalpower<(630/3)*totalturbs and len(AFFturb)>0:
      for _ in range(min(len(AFFturb),10)):
        moveturb=rng.choice(AFFturb)
        newpos=rng.choice(poschoices)
        if powergrid.ravel()[newpos]==0:
          g.ravel()[newpos]=1
          g.ravel()[moveturb]=0
    # Recalculate output values of new g:
    g,powergrid,totalpower,totalturbs=output(g)
    return g.flatten()

class CustomCrossover(Crossover):
  def __init__(self,prob=1.0,eta=5,problem=None):
    super().__init__(2, 2) # for 2 parents, 2 offspring

  def _do(self,problem,X,algorithm,**kwargs):
    n_parents,n_matings,n_var=X.shape
    args=[]
    # Prepare input for each pair of parents:
    for k in range(n_matings):
      parents=X[:,k] # for shape (2,n_var)
      parenta=parents[0]
      parentb=parents[1]
      idx=k*2
      args.append((parenta,parentb,idx,problem.n_var,algorithm.n_gen,algorithm.pop))
    # Parallel crossover function per pair of parents:
    with ProcessPoolExecutor() as executor:
      results=list(executor.map(self.crosspair,args))
    # Convert results into array:
    Y=np.full_like(X,None,dtype=object)
    # Extract offspring from each parental-mating:
    for k,(offa,offb) in enumerate(results):
        Y[0,k]=offa
        Y[1,k]=offb
    return Y

  def crosspair(self,args):
    a,b,idx,n_var,n_gen,pop=args
    # Initialise offspring:
    offa=np.zeros(n_var,dtype=object)
    offb=np.zeros(n_var,dtype=object)
    if n_gen>2:
      aFV=pop[idx].get("F")
      bFV=pop[idx+1].get("F")
      for i in range(n_var):
        # Compare first objective
        if aFV[0]<bFV[0]:
          offa[i]=a[i]
          offb[i]=a[i]
        elif bFV[0]<aFV[0]:
          offa[i]=b[i]
          offb[i]=b[i]
        # Compare second objective
        elif aFV[1]<bFV[1]:
          offa[i]=a[i]
          offb[i]=a[i]
        elif bFV[1]<aFV[1]:
          offa[i]=b[i]
          offb[i]=b[i]
        else:
          if i%2==0:
            offa[i]=b[n_var - 1 - i]
            offb[i]=a[n_var - 1 - i]
          else:
            offa[i]=b[i]
            offb[i]=a[i]
    else:
      for i in range(n_var):
        if i%2==0:
          offa[i]=b[i]
          offb[i]=a[i]
        else:
          offa[i]=a[i]
          offb[i]=b[i]
    return offa,offb

def indeval(X):
  # Calculate the output values for individual solutions, X:
  X=X.reshape((10, 10))
  g,powergrid,totalpower,totalturbs=output(X)
  return [-totalpower,totalturbs]

class popeval(Evaluator):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def _eval(self,problem,pop,evaluate_values_of=None,**kwargs):
    allX=[ind.X for ind in pop]
    # Calculate function values using parallel processing:
    with ProcessPoolExecutor() as executor:
      funcvals=list(executor.map(indeval,allX))
    for i,funcval in enumerate(funcvals):
      pop[i].F=funcval # store results for all solutions in the current population

  def _Xeval(self,problem,X):
    out={} # create "out" dictionary
    problem._evaluate(X,out)
    return out["F"]

class OptimisationProblem(ElementwiseProblem):
  def __init__(self,algorithm=None):
    # Define optimisation problem as an elementwise problem:
    super().__init__(n_var=100,n_obj=2,n_ieq_constr=1,n_eq_constr=0,xl=0,xu=1)
    self.algorithm=algorithm # store algorithm
    self.midturbs=16 # initiate midturbs value

  def _evaluate(self,x,out):
    g,powergrid,totalpower,totalturbs=output(x.reshape(10,10))
    originalturbs=np.sum(g) # Number of turbines before considering wake effects
    # Prioritise maximisation of totalpower over minimisation of totalturbs:
    f1weight=1.0
    f2weight=0.5
    f1=-totalpower*f1weight # maximize overall power output
    f2=totalturbs*f2weight # minimize turbine count
    # Add optimisation functions to "out"-dictionary
    out["F"]=[f1,f2]
    # Prioritise the constraint of number of turbines over the constraint of turbine distances:
    g1weight=1.0
    g2weight=0.5
    g1=(originalturbs-self.midturbs)*g1weight # number of turbines
    # Inequality constraint for turbine distances:
    turbs=np.argwhere(powergrid>0) # positions of turbines
    turbdists=eucdistALL[np.ix_(turbs.flatten(),turbs.flatten())] # extract pairwise distances of all turbines
    np.fill_diagonal(turbdists,np.inf) # discard diagonal entries
    g2=3-np.min(turbdists)*g2weight # encourage pairwise distances of all turbines > 3
    # Add inequality constraint function to "out"-dictionary:
    out["G"]=[g1,g2]


class CustomCallback(Callback):
  def __init__(self,maxgen,startturbs=30,endturbs=16):
    super().__init__()
    self.data["BestFV"]=[]
    self.data["BestTurbineNumber"]=[]
    self.maxgen=maxgen
    self.startturbs=startturbs
    self.endturbs=endturbs

  def notify(self,algorithm):
    F=algorithm.pop.get("F") # assign function values to variable
    indexbestf1=np.argmin(F[:,0])
    self.data["BestFV"].append(F[indexbestf1,0])
    self.data["BestTurbineNumber"].append(F[indexbestf1,1])
    # Calculate the current generation as a percentage of the max generations:
    genperc=algorithm.n_gen/self.maxgen
    # Adjust midturbs value accordingly:
    midturbs=int(self.startturbs-(self.startturbs-self.endturbs)*genperc)
    # Adjust the constraint:
    problem=algorithm.problem # store problem
    problem.midturbs=midturbs
    print(f'Generation {algorithm.n_gen} has completed.') # track algorithm progress

def printres(res,algorithm):
  # Define algorithm name:
  algname="NSGA2" if isinstance(algorithm,NSGA2) else "SPEA2"
  # Get the time:
  time=datetime.datetime.now().strftime("%H%M%S")

  # Plot and print the results for the optimum solution:
  soln=np.reshape(np.asarray(res.opt.get("X")[0]),(10,10)) # 'best' grid layout
  coords=np.swapaxes((np.argwhere(soln)),0,1)
  solnX=coords[0]
  solnY=coords[1]
  SolnOutput=output(soln)
  fig3,ax4=plt.subplots(1,1)
  ax4.set_aspect('equal',adjustable='box')
  ax4.set_xlim(-1,10)
  ax4.set_ylim(-1,10)
  ax4.set_title(f'Best turbine layout found for {algname}:')
  ax4.set_xticks(np.arange(0,10,1))
  ax4.set_yticks(np.arange(0,10,1))
  ax4.grid(visible=True,which='both',axis='both',color='grey',linestyle='-',linewidth=1)
  plt.scatter(solnX,solnY,edgecolors='mediumvioletred',facecolors='grey')
  plt.show()
  plt.savefig(f'turbinesbestlayout{algname}{time}.png')
  files.download(f'turbinesbestlayout{algname}{time}.png')
  plt.close(fig3)
  print(f'\nBest solution for {algname}: ')
  print('\nPower output representation per turbine (kWh): \n',np.round(SolnOutput[1],0))
  print('\nTotal power output (kWh): ',-res.F[0][0])
  print('\nNumber of turbines: ',res.algorithm.callback.data["BestTurbineNumber"][0])

  # Plot function values(/max. power outputs) Vs. generation number:
  FV=res.algorithm.callback.data["BestFV"]
  FV=(np.asarray(FV)).flatten()
  Power=[-1*FV for FV in FV]
  Power=np.asarray(Power).flatten()
  fig1,(ax1,ax2)=plt.subplots(2,1)
  ax1.set_title(f'Best function value at each generation for {algname}:')
  ax1.set_xlabel('Generation number')
  ax1.xaxis.set_major_locator(MaxNLocator(nbins='auto',integer=True))
  ax1.set_ylabel('Function value')
  ax1.yaxis.set_major_locator(MaxNLocator(nbins='auto'))
  ax1.plot(np.arange(len(FV)),FV,color='darkmagenta')
  ax2.set_title(f'Maximum overall power output at each generation for {algname}:')
  ax2.set_xlabel('Generation number')
  ax2.xaxis.set_major_locator(MaxNLocator(nbins='auto',integer=True))
  ax2.set_ylabel('Power output (kWh)')
  ax2.yaxis.set_major_locator(MaxNLocator(nbins='auto'))
  ax2.plot(np.arange(len(FV)),Power,color='blueviolet')
  fig1.tight_layout()
  plt.show()
  plt.savefig(f'GenerationsBestFV{algname}{time}.png')
  files.download(f'GenerationsBestFV{algname}{time}.png')
  plt.close(fig1)

  # Plot number of turbines Vs. power output:
  fig4,ax5=plt.subplots(1,1)
  turbines=res.algorithm.callback.data["BestTurbineNumber"]
  # Use jitter to show duplicate values on plot:
  jitter=0.1
  turbsjittered=[t+random.uniform(-jitter,jitter) for t in turbines]
  Powerjittered=[p+random.uniform(-jitter,jitter) for p in Power]
  ax5.set_title(f'Turbines Vs. Power Output for {algname}')
  ax5.set_xlabel('Number of turbines')
  ax5.set_ylabel('Total power output')
  ax5.xaxis.set_major_locator(MaxNLocator(nbins='auto',integer=True))
  ax5.yaxis.set_major_locator(MaxNLocator(nbins='auto'))
  plt.scatter(turbsjittered, Powerjittered, edgecolor='blueviolet', facecolor='none')
  plt.show()
  plt.savefig(f'TurbinesVsPower{algname}{time}.png')
  files.download(f'TurbinesVsPower{algname}{time}.png')
  plt.close()

In [ ]:
# Solving with NSGA2:
Algorithm_NSGA2=CustomNSGA2(pop_size=50, # number of parents in each generation
          sampling=CustomSampling(), # custom sampling method
          mutation=CustomMutation(prob=1.0,eta=5), # custom mutation method
          crossover=CustomCrossover(prob=1.0,eta=5), # custom crossover method
          callback=CustomCallback(maxgen=20), # custom callback method
          eliminate_duplicates=True)

res1=minimize(problem=OptimisationProblem(algorithm=Algorithm_NSGA2),
            algorithm=Algorithm_NSGA2,
            evaluator=popeval(),
            termination=('n_gen',20), # terminate after 100 generations
            seed=None,
            verbose=False,
            save_history=False)

# Printing 'best' solution of NSGA2:
if res1.opt is not None:
  printres(res1,Algorithm_NSGA2)
else:
  print("The NSGA2 algorithm did not find a feasible solution.")

In [ ]:
# Solving with SPEA2:
Algorithm_SPEA2=CustomSPEA2(pop_size=200, # number of parents in each generation
          sampling=CustomSampling(), # custom sampling method
          mutation=CustomMutation(prob=1.0,eta=5), # custom mutation method
          crossover=CustomCrossover(prob=1.0,eta=5), # custom crossover method
          callback=CustomCallback(maxgen=30), # custom callback method
          eliminate_duplicates=True)

res2=minimize(problem=OptimisationProblem(algorithm=Algorithm_SPEA2),
            algorithm=Algorithm_SPEA2,
            evaluator=popeval(),
            termination=('n_gen',30), # terminate after 100 generations
            seed=None,
            verbose=False,
            save_history=False)

# Printing 'best' solution of SPEA2:
if res2.opt is not None:
  printres(res2,Algorithm_SPEA2)
else:
  print("The SPEA2 algorithm did not find a feasible solution.")

# Old output(G,ab)  
Not intended to produce useable output. This is the old version of my wake model, referenced in my project report.

In [ ]:
def OldOutput(G,ab):
  import numpy as np
  from numpy import random as rng
  import math

  p=np.argwhere(G) # stores coordinates of turbine placements
  Disp=[] # creates an empty list in which to store the displacements between each turbine

  # Defining the variables used to calculate wake region R in terms of Coordinate Units (CU):

  z=0.06 # (CU) height of wind turbine
  z0=0.0003 # (CU) roughness of turain
  k=0.5/math.log(z/z0)

  r0=0.02 # (CU) rotor radius
  a=0.3267949192 # axial induction factor
  r=r0*math.sqrt((1-a)/(1-2*a))

  x=1 # (CU) this variable was not defined in the paper

  R=(k*x)+r # wake region
  RCU=R # converts the wake region from meters into coordinate units (CU) [redundant]

  if len(p)>1:
    P=np.swapaxes(p,0,1) # separates the x- and y-coordinates into two axes for multiple turbines
  else:
    P=p # case for single turbine

  # Creating empty lists in which to store the coordinates of turbine positions:
  row0=[]
  row1=[]
  row2=[]
  row3=[]
  row4=[]
  row5=[]
  row6=[]
  row7=[]
  row8=[]
  row9=[]

  # Separating the coordinates from each row of the original grid into their respective lists:
  # Case for single turbine:
  if len(p)==1:
    for i in range (0,len(P[0])):
      if P[0][i]==0:
          row0.append(p[i])
      elif P[0][i]==1:
          row1.append(p[i])
      elif P[0][i]==2:
          row2.append(p[i])
      elif P[0][i]==3:
          row3.append(p[i])
      elif P[0][i]==4:
          row4.append(p[i])
      elif P[0][i]==5:
          row5.append(p[i])
      elif P[0][i]==6:
          row6.append(p[i])
      elif P[0][i]==7:
          row7.append(p[i])
      elif P[0][i]==8:
          row8.append(p[i])
      elif P[0][i]==9:
          row9.append(p[i])
      else:
          print('error')
          break
  # Case for multiple turbines:
  else:
    for i in range (0,len(P[0])):
      if P[0,i]==0:
          row0.append(p[i])
      elif P[0,i]==1:
          row1.append(p[i])
      elif P[0,i]==2:
          row2.append(p[i])
      elif P[0,i]==3:
          row3.append(p[i])
      elif P[0,i]==4:
          row4.append(p[i])
      elif P[0,i]==5:
          row5.append(p[i])
      elif P[0,i]==6:
          row6.append(p[i])
      elif P[0,i]==7:
          row7.append(p[i])
      elif P[0,i]==8:
          row8.append(p[i])
      elif P[0,i]==9:
          row9.append(p[i])
      else:
        print('error')
        break

  # Swapping axes so the individual x-coordinates can be referred to later:
  row0=np.swapaxes(row0, 0, 1) if len(row0) > 1 else row0
  row1=np.swapaxes(row1, 0, 1) if len(row1) > 1 else row1
  row2=np.swapaxes(row2, 0, 1) if len(row2) > 1 else row2
  row3=np.swapaxes(row3, 0, 1) if len(row3) > 1 else row3
  row4=np.swapaxes(row4, 0, 1) if len(row4) > 1 else row4
  row5=np.swapaxes(row5, 0, 1) if len(row5) > 1 else row5
  row6=np.swapaxes(row6, 0, 1) if len(row6) > 1 else row6
  row7=np.swapaxes(row7, 0, 1) if len(row7) > 1 else row7
  row8=np.swapaxes(row8, 0, 1) if len(row8) > 1 else row8
  row9=np.swapaxes(row9, 0, 1) if len(row9) > 1 else row9

  # Restricting the lists to include only the x-coordinates:
  Row0 = row0[1] if len(row0) > 1 else []
  Row1 = row1[1] if len(row1) > 1 else []
  Row2 = row2[1] if len(row2) > 1 else []
  Row3 = row3[1] if len(row3) > 1 else []
  Row4 = row4[1] if len(row4) > 1 else []
  Row5 = row5[1] if len(row5) > 1 else []
  Row6 = row6[1] if len(row6) > 1 else []
  Row7 = row7[1] if len(row7) > 1 else []
  Row8 = row8[1] if len(row8) > 1 else []
  Row9 = row9[1] if len(row9) > 1 else []


  # Combining all rows into an overall list:
  ROWS=[Row0,Row1,Row2,Row3,Row4,Row5,Row6,Row7,Row8,Row9]

  # Creating empty lists in which to store the x-axis distance between turbines:
  Distances0=[]
  Distances1=[]
  Distances2=[]
  Distances3=[]
  Distances4=[]
  Distances5=[]
  Distances6=[]
  Distances7=[]
  Distances8=[]

  Diag=[] # creating an empty list to store the distance between one entry in row h and the entirity of the next row

  for h in range (0,9): # calculates the distances between turbines in adjacent rows of the grid
      row=ROWS[h] # allows individual x-coordinates to be referenced

      for j in range (0,len(row)):
          entry=row[j]
          line2=ROWS[h+1]
          Diagonal=np.subtract.outer(line2,entry) # calculates the distance between one entry and all entries in the next row
          Diag.append(Diagonal) # temporarily adds these distances to the list 'Diag'

      # Add the distances calculated into the relevant list:
      if h==0:
          Distances0.append(Diag)
          Diag=[]
      elif h==1:
          Distances1.append(Diag)
          Diag=[]
      elif h==2:
          Distances2.append(Diag)
          Diag=[]
      elif h==3:
          Distances3.append(Diag)
          Diag=[]
      elif h==4:
          Distances4.append(Diag)
          Diag=[]
      elif h==5:
          Distances5.append(Diag)
          Diag=[]
      elif h==6:
          Distances6.append(Diag)
          Diag=[]
      elif h==7:
          Distances7.append(Diag)
          Diag=[]
      elif h==8:
          Distances8.append(Diag)
          Diag=[]
      else:
          print('error')
          break

  # Getting rid of an unecessary 0-th axis:
  Distances0=Distances0[0]
  Distances1=Distances1[0]
  Distances2=Distances2[0]
  Distances3=Distances3[0]
  Distances4=Distances4[0]
  Distances5=Distances5[0]
  Distances6=Distances6[0]
  Distances7=Distances7[0]
  Distances8=Distances8[0]

  DistancesALL=[Distances0,Distances1,Distances2,Distances3,Distances4,Distances5,Distances6,Distances7,Distances8] # Creates an overall list of all distances

  # Defining empty lists for each row of distances between each element of each row:
  distances00=[]
  distances01=[]
  distances02=[]
  distances03=[]
  distances04=[]
  distances05=[]
  distances06=[]
  distances07=[]
  distances08=[]
  distances09=[]
  distances10=[]
  distances11=[]
  distances12=[]
  distances13=[]
  distances14=[]
  distances15=[]
  distances16=[]
  distances17=[]
  distances18=[]
  distances19=[]
  distances20=[]
  distances21=[]
  distances22=[]
  distances23=[]
  distances24=[]
  distances25=[]
  distances26=[]
  distances27=[]
  distances28=[]
  distances29=[]
  distances30=[]
  distances31=[]
  distances32=[]
  distances33=[]
  distances34=[]
  distances35=[]
  distances36=[]
  distances37=[]
  distances38=[]
  distances39=[]
  distances40=[]
  distances41=[]
  distances42=[]
  distances43=[]
  distances44=[]
  distances45=[]
  distances46=[]
  distances47=[]
  distances48=[]
  distances49=[]
  distances50=[]
  distances51=[]
  distances52=[]
  distances53=[]
  distances54=[]
  distances55=[]
  distances56=[]
  distances57=[]
  distances58=[]
  distances59=[]
  distances50=[]
  distances51=[]
  distances52=[]
  distances53=[]
  distances54=[]
  distances55=[]
  distances56=[]
  distances57=[]
  distances58=[]
  distances59=[]
  distances60=[]
  distances61=[]
  distances62=[]
  distances63=[]
  distances64=[]
  distances65=[]
  distances66=[]
  distances67=[]
  distances68=[]
  distances69=[]
  distances70=[]
  distances71=[]
  distances72=[]
  distances73=[]
  distances74=[]
  distances75=[]
  distances76=[]
  distances77=[]
  distances78=[]
  distances79=[]
  distances80=[]
  distances81=[]
  distances82=[]
  distances83=[]
  distances84=[]
  distances85=[]
  distances86=[]
  distances87=[]
  distances88=[]
  distances89=[]

  # Separating the array of distances between each element and the row below:
  for i in range (0,len(Distances0)):
      if i==0:
          distances00.append(Distances0[0])
          distances00=distances00[0]
      elif i==1:
          distances01.append(Distances0[1])
          distances01=distances01[0]
      elif i==2:
          distances02.append(Distances0[2])
          distances02=distances02[0]
      elif i==3:
          distances03.append(Distances0[3])
          distances03=distances03[0]
      elif i==4:
          distances04.append(Distances0[4])
          distances04=distances04[0]
      elif i==5:
          distances05.append(Distances0[5])
          distances05=distances05[0]
      elif i==6:
          distances06.append(Distances0[6])
          distances06=distances06[0]
      elif i==7:
          distances07.append(Distances0[7])
          distances07=distances07[0]
      elif i==8:
          distances08.append(Distances0[8])
          distances08=distances08[0]
      elif i==9:
          distances09.append(Distances0[9])
          distances09=distances09[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances1)):
      if i==0:
          distances10.append(Distances1[0])
          distances10=distances10[0]
      elif i==1:
          distances11.append(Distances1[1])
          distances11=distances11[0]
      elif i==2:
          distances12.append(Distances1[2])
          distances12=distances12[0]
      elif i==3:
          distances13.append(Distances1[3])
          distances13=distances13[0]
      elif i==4:
          distances14.append(Distances1[4])
          distances14=distances14[0]
      elif i==5:
          distances15.append(Distances1[5])
          distances15=distances15[0]
      elif i==6:
          distances16.append(Distances1[6])
          distances16=distances16[0]
      elif i==7:
          distances17.append(Distances1[7])
          distances17=distances17[0]
      elif i==8:
          distances18.append(Distances1[8])
          distances18=distances18[0]
      elif i==9:
          distances19.append(Distances1[9])
          distances19=distances19[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances2)):
      if i==0:
          distances20.append(Distances2[0])
          distances20=distances20[0]
      elif i==1:
          distances21.append(Distances2[1])
          distances21=distances21[0]
      elif i==2:
          distances22.append(Distances2[2])
          distances22=distances22[0]
      elif i==3:
          distances23.append(Distances2[3])
          distances23=distances23[0]
      elif i==4:
          distances24.append(Distances2[4])
          distances24=distances24[0]
      elif i==5:
          distances25.append(Distances2[5])
          distances25=distances25[0]
      elif i==6:
          distances26.append(Distances2[6])
          distances26=distances26[0]
      elif i==7:
          distances27.append(Distances2[7])
          distances27=distances27[0]
      elif i==8:
          distances28.append(Distances2[8])
          distances28=distances28[0]
      elif i==9:
          distances29.append(Distances2[9])
          distances29=distances29[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances3)):
      if i==0:
          distances30.append(Distances3[0])
          distances30=distances30[0]
      elif i==1:
          distances31.append(Distances3[1])
          distances31=distances31[0]
      elif i==2:
          distances32.append(Distances3[2])
          distances32=distances32[0]
      elif i==3:
          distances33.append(Distances3[3])
          distances33=distances33[0]
      elif i==4:
          distances34.append(Distances3[4])
          distances34=distances34[0]
      elif i==5:
          distances35.append(Distances3[5])
          distances35=distances35[0]
      elif i==6:
          distances36.append(Distances3[6])
          distances36=distances36[0]
      elif i==7:
          distances37.append(Distances3[7])
          distances37=distances37[0]
      elif i==8:
          distances38.append(Distances3[8])
          distances38=distances38[0]
      elif i==9:
          distances39.append(Distances3[9])
          distances39=distances39[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances4)):
      if i==0:
          distances40.append(Distances4[0])
          distances40=distances40[0]
      elif i==1:
          distances41.append(Distances4[1])
          distances41=distances41[0]
      elif i==2:
          distances42.append(Distances4[2])
          distances42=distances42[0]
      elif i==3:
          distances43.append(Distances4[3])
          distances43=distances43[0]
      elif i==4:
          distances44.append(Distances4[4])
          distances44=distances44[0]
      elif i==5:
          distances45.append(Distances4[5])
          distances45=distances45[0]
      elif i==6:
          distances46.append(Distances4[6])
          distances46=distances46[0]
      elif i==7:
          distances47.append(Distances4[7])
          distances47=distances47[0]
      elif i==8:
          distances48.append(Distances4[8])
          distances48=distances48[0]
      elif i==9:
          distances49.append(Distances4[9])
          distances49=distances49[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances5)):
      if i==0:
          distances50.append(Distances5[0])
          distances50=distances50[0]
      elif i==1:
          distances51.append(Distances5[1])
          distances51=distances51[0]
      elif i==2:
          distances52.append(Distances5[2])
          distances52=distances52[0]
      elif i==3:
          distances53.append(Distances5[3])
          distances53=distances53[0]
      elif i==4:
          distances54.append(Distances5[4])
          distances54=distances54[0]
      elif i==5:
          distances55.append(Distances5[5])
          distances55=distances55[0]
      elif i==6:
          distances56.append(Distances5[6])
          distances56=distances56[0]
      elif i==7:
          distances57.append(Distances5[7])
          distances57=distances57[0]
      elif i==8:
          distances58.append(Distances5[8])
          distances58=distances58[0]
      elif i==9:
          distances59.append(Distances5[9])
          distances59=distances59[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances6)):
      if i==0:
          distances60.append(Distances6[0])
          distances60=distances60[0]
      elif i==1:
          distances61.append(Distances6[1])
          distances61=distances61[0]
      elif i==2:
          distances62.append(Distances6[2])
          distances62=distances62[0]
      elif i==3:
          distances63.append(Distances6[3])
          distances63=distances63[0]
      elif i==4:
          distances64.append(Distances6[4])
          distances64=distances64[0]
      elif i==5:
          distances65.append(Distances6[5])
          distances65=distances65[0]
      elif i==6:
          distances66.append(Distances6[6])
          distances66=distances66[0]
      elif i==7:
          distances67.append(Distances6[7])
          distances67=distances67[0]
      elif i==8:
          distances68.append(Distances6[8])
          distances68=distances68[0]
      elif i==9:
          distances69.append(Distances6[9])
          distances69=distances69[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances7)):
      if i==0:
          distances70.append(Distances7[0])
          distances70=distances70[0]
      elif i==1:
          distances71.append(Distances7[1])
          distances71=distances71[0]
      elif i==2:
          distances72.append(Distances7[2])
          distances72=distances72[0]
      elif i==3:
          distances73.append(Distances7[3])
          distances73=distances73[0]
      elif i==4:
          distances74.append(Distances7[4])
          distances74=distances74[0]
      elif i==5:
          distances75.append(Distances7[5])
          distances75=distances75[0]
      elif i==6:
          distances76.append(Distances7[6])
          distances76=distances76[0]
      elif i==7:
          distances77.append(Distances7[7])
          distances77=distances77[0]
      elif i==8:
          distances78.append(Distances7[8])
          distances78=distances78[0]
      elif i==9:
          distances79.append(Distances7[9])
          distances79=distances79[0]
      else:
          print('error')
          break

  for i in range (0,len(Distances8)):
      if i==0:
          distances80.append(Distances8[0])
          distances80=distances80[0]
      elif i==1:
          distances81.append(Distances8[1])
          distances81=distances81[0]
      elif i==2:
          distances82.append(Distances8[2])
          distances82=distances82[0]
      elif i==3:
          distances83.append(Distances8[3])
          distances83=distances83[0]
      elif i==4:
          distances84.append(Distances8[4])
          distances84=distances84[0]
      elif i==5:
          distances85.append(Distances8[5])
          distances85=distances85[0]
      elif i==6:
          distances86.append(Distances8[6])
          distances86=distances86[0]
      elif i==7:
          distances87.append(Distances8[7])
          distances87=distances87[0]
      elif i==8:
          distances88.append(Distances8[8])
          distances88=distances88[0]
      elif i==9:
          distances89.append(Distances8[9])
          distances89=distances89[0]
      else:
          print('error')
          break

  # Defining lists for those turbines affected by wake:
  ATR0=[]
  ATR1=[]
  ATR2=[]
  ATR3=[]
  ATR4=[]
  ATR5=[]
  ATR6=[]
  ATR7=[]
  ATR8=[]

  # Adding the distances between each element with the elements in the row below to a corresponding list:
  for i in range (0,len(Row0)):
      for j in range (0,len(Row1)):
          if i==0:
              if abs(distances00[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances00[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances01[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances01[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances02[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances02[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances03[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances03[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances04[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances04[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances05[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances05[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances06[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances06[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances07[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances07[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances08[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances08[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances09[j])<=RCU:
                  ATR0.append([Row0[i],Row1[j],distances09[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row1)):
      for j in range (0,len(Row2)):
          if i==0:
              if abs(distances10[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances10[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances11[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances11[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances12[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances12[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances13[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances13[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances14[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances14[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances15[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances15[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances16[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances16[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances17[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances17[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances18[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances18[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances19[j])<=RCU:
                  ATR1.append([Row1[i],Row2[j],distances19[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row2)):
      for j in range (0,len(Row3)):
          if i==0:
              if abs(distances20[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances20[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances21[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances21[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances22[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances22[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances23[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances23[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances24[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances24[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances25[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances25[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances26[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances26[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances27[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances27[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances28[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances28[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances29[j])<=RCU:
                  ATR2.append([Row2[i],Row3[j],distances29[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row3)):
      for j in range (0,len(Row4)):
          if i==0:
              if abs(distances30[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances30[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances31[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances31[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances32[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances32[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances33[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances33[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances34[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances34[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances35[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances35[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances36[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances36[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances37[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances37[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances38[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances38[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances39[j])<=RCU:
                  ATR3.append([Row3[i],Row4[j],distances39[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row4)):
      for j in range (0,len(Row5)):
          if i==0:
              if abs(distances40[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances40[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances41[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances41[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances42[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances42[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances43[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances43[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances44[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances44[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances45[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances45[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances46[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances46[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances47[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances47[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances48[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances48[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances49[j])<=RCU:
                  ATR4.append([Row4[i],Row5[j],distances49[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row5)):
      for j in range (0,len(Row6)):
          if i==0:
              if abs(distances50[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances50[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances51[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances51[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances52[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances52[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances53[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances53[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances54[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances54[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances55[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances55[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances56[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances56[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances57[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances57[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances58[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances58[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances59[j])<=RCU:
                  ATR5.append([Row5[i],Row6[j],distances59[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row6)):
      for j in range (0,len(Row7)):
          if i==0:
              if abs(distances60[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances60[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances61[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances61[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances62[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances62[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances63[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances63[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances64[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances64[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances65[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances65[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances66[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances66[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances67[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances67[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances68[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances68[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances69[j])<=RCU:
                  ATR6.append([Row6[i],Row7[j],distances69[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row7)):
      for j in range (0,len(Row8)):
          if i==0:
              if abs(distances70[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances70[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances71[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances71[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances72[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances72[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances73[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances73[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances74[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances74[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances75[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances75[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances76[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances76[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances77[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances77[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances78[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances78[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances79[j])<=RCU:
                  ATR7.append([Row7[i],Row8[j],distances79[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  for i in range (0,len(Row8)):
      for j in range (0,len(Row9)):
          if i==0:
              if abs(distances80[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances80[j]])
                  continue
              else:
                  continue
          if i==1:
              if abs(distances81[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances81[j]])
                  continue
              else:
                  continue
          if i==2:
              if abs(distances82[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances82[j]])
                  continue
              else:
                  continue
          if i==3:
              if abs(distances83[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances83[j]])
                  continue
              else:
                  continue
          if i==4:
              if abs(distances84[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances84[j]])
                  continue
              else:
                  continue
          if i==5:
              if abs(distances85[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances85[j]])
                  continue
              else:
                  continue
          if i==6:
              if abs(distances86[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances86[j]])
                  continue
              else:
                  continue
          if i==7:
              if abs(distances87[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances87[j]])
                  continue
              else:
                  continue
          if i==8:
              if abs(distances88[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances88[j]])
                  continue
              else:
                  continue
          if i==9:
              if abs(distances89[j])<=RCU:
                  ATR8.append([Row8[i],Row9[j],distances89[j]])
                  continue
              else:
                  continue
          else:
              print('error')
              break

  ATRALL=[ATR0,ATR1,ATR2,ATR3,ATR4,ATR5,ATR6,ATR7,ATR8]

  # Creating a grid for positions of turbines affected by wake:
  GridA=np.zeros(shape=(10,10),dtype=np.int_)
  GridA[0]=0

  for i in range (0,len(ATR0)):
      a=ATR0[i]
      x=a[0]
      GridA[1,x]=1

  for i in range (0,len(ATR1)):
      a=ATR1[i]
      x=a[0]
      GridA[2,x]=1

  for i in range (0,len(ATR2)):
      a=ATR2[i]
      x=a[0]
      GridA[3,x]=1

  for i in range (0,len(ATR3)):
      a=ATR3[i]
      x=a[0]
      GridA[4,x]=1

  for i in range (0,len(ATR4)):
      a=ATR4[i]
      x=a[0]
      GridA[5,x]=1

  for i in range (0,len(ATR5)):
      a=ATR5[i]
      x=a[0]
      GridA[6,x]=1

  for i in range (0,len(ATR6)):
      a=ATR6[i]
      x=a[0]
      GridA[7,x]=1

  for i in range (0,len(ATR7)):
      a=ATR7[i]
      x=a[0]
      GridA[8,x]=1

  for i in range (0,len(ATR8)):
      a=ATR8[i]
      x=a[0]
      GridA[9,x]=1

  # Repeating the above process, with vastly simpler code, now that variables are defined such that they are easier to work with:
  atrallIN=[] # creates a temporary 'holding' place for arrays after the effects of wake are assessed
  atrallOUT=[] # creates a temporary 'holding' place for arrays after the effects of wake are assessed
  coords=[] # creates a temporary 'holding' place for coordinates of turbines affected by wake
  Grids=[] # stores each grid throughout the iterative process of assessing the effects of wake
  Grids.append(np.asarray(G,dtype=np.int_)) # add the original layout array 'L' to the 'Grids' list
  Grids.append(np.asarray(GridA,dtype=np.int_)) # add the first array after assessing the effects of wake to the 'Grids' list

  for j in range (0,len(ATRALL)-1):
      ATRrow1=ATRALL[j]
      ATRrow2=ATRALL[j+1]

      for k in range (0,len(ATRrow1)):
          a=ATRrow1[k]

          for l in range (0,len(ATRrow2)):
              b=ATRrow2[l]

              # Calculating the horizontal distance between coordinates and comparing to the value for the wake region:
              if abs(a[1]-b[1])<=RCU:
                  c=[a[1],j+1]
                  atrallIN.append(c)
                  coords.append([j+2,b[1]])
                  continue
              else:
                  continue

  Grids.append(np.asarray(np.zeros(shape=(10,10),dtype=np.int_))) # adding an zeros array to the 'Grids' list as a placeholder to which values will be assigned from list 'coords'
  GridB=Grids[2]
  for trb in range (0,len(coords)):
      s,t=coords[trb] # convert tuples in list 'coordsA' to coordinates
      GridB[s,t]=1 # assign the coordinates from list 'afftrb' to equal one in the new grid

  coords=[] # empty the 'coords' list
  k=3 # initiate variable 'k' corresponding to the list element in 'Grids'

  while len(atrallIN)>1:

      for i in range (0,len(atrallIN)):
          a=atrallIN[i]

          for j in range (0,len(atrallIN)):

              if i<j and j!=i:
                  b=atrallIN[j]
                  # Calculating the horizontal distance between coordinates and comparing to the value for the wake region:
                  c=a[0]-b[0]
                  if abs(c)<=RCU:
                      d=[a[0],a[1]]
                      if b[1]==a[1]+1:
                          atrallOUT.append(d)
                          coords.append([b[1]+1,b[0]])
                      else:
                          continue
                      continue
                  else:
                      continue
                  continue
              else:
                  continue

      Grids.append(np.asarray(np.zeros(shape=(10,10),dtype=np.int_)))
      GridC=Grids[k]
      for trb in range (0,len(coords)):
          s,t=coords[trb] # convert tuples in list 'coordsA' to coordinates
          GridC[s,t]=1 # assign the coordinates from list 'afftrb' to equal one in the new grid

      coords=[] # empty the 'coords' list
      atrallIN=[] # empty the 'atrallIN' list
      k=k+1 # increase 'k' corresponding to the list element in 'Grids'

      for i in range (0,len(atrallOUT)):
          a=atrallOUT[i]

          for j in range (0,len(atrallOUT)):

              if i<j and j!=i:
                  b=atrallOUT[j]
                  # Calculating the horizontal distance between coordinates and comparing to the value for the wake region:
                  c=a[0]-b[0]
                  if abs(c)<=RCU:
                      d=[a[0],a[1]]
                      if b[1]==a[1]+1:
                          atrallIN.append(d)
                          coords.append([b[1]+1,b[0]])
                          continue
                      else:
                          continue
                  else:
                      continue
                  continue
              else:
                  continue

      Grids.append(np.asarray(np.zeros(shape=(10,10),dtype=np.int_)))
      GridD=Grids[k]
      for trb in range (0,len(coords)):
          s,t=coords[trb] # convert tuples in list 'coordsA' to coordinates
          GridD[s,t]=1 # assign the coordinates from list 'afftrb' to equal one in the new grid

      coords=[] # empty the 'coords' list
      atrallOUT=[] # empty the 'atrallOUT' list
      k=k+1 # increase 'k' corresponding to the list element in 'Grids'

  pwr=[] # creates an empty list to store values of velocity behind each wind turbine
  pwr.append(Grids[0]*630) # adds the initial grid layout to the pwr list and multiplies it by the rated output of the model turbine in kWh
  sum=pwr[0] # initiates the value of sum to equal the first entry in the pwr list
  v=[] # creates an empty list to store the v and v0 values associated with each pass of the for loop
  Ct=0.88 # defines the thrust coefficient
  k=0.0943695829 # defines wake decay factor
  x=1 # vertical distance (CU) between turbines
  a=0.3267949192 # defines axial induction factor
  r0=0.02 # defines rotor radius
  r=r0*(math.sqrt((1-a)/(1-2*a))) # calculates and defines the wake region immediately behind the rotor
  v.append(630) # sets the first entry of the v list to equal 630 (the rated power output of the model turbine)
  j=1 # initiates the value j

  for i in range (1,len(Grids)-1): # for the number of grids in the Grids list
      v.append(v[j-1]*(1-(1-math.sqrt(1-Ct))/math.pow((1+(k*x)/r),2))) # add this calculation for v to the v list
      pwr.append(Grids[i]*v[j]) # add to the pwr list, the i-th entry of the Grids list with its entries multiplied by the above value of v
      j=j+1 # iterate value j
      sum=sum-pwr[i] # subtract the pwr array calculated above from the initial pwr array, pwr[0]

  pwrall=np.asarray(sum) # create an array of the power outputs from each turbine

  pwrALL=np.argwhere(pwrall>0) # find all negtive values in the above array
  PWRALL=np.zeros(shape=(10,10)) # generate a 10x10 array of zeros

  # Convert list of coordinates where values are negative into a matrix where those coordinates equal 1:
  for i in range (0,len(pwrALL)):
        s,t=pwrALL[i]
        PWRALL[s,t]=1

  PWRALL=abs(PWRALL*sum) # multiply the above matrix by the array of power outputs without decimals

  power=np.sum(pwrall)

  return pwrall,PWRALL,power, G

# Gekko optimisation  
Not intended to produce useable output. This is the implementation of my optimisation problem, referenced in my project report.

In [ ]:
def discoptpwr1(T):
  if isinstance(T,int) and 0<T<99:
    import tempfile
    import os
    import shutil
    import pandas as pd
    import numpy as np
    from numpy import random as rng

    temp_dir=tempfile.mkdtemp() # Create temporary directory for results

    try:
      m=GEKKO(remote=True)
      m.path=temp_dir # Assign location of temporary directory

      G=layoutgen(T)[0]
      ab=layoutgen(T)[1]
      R=output(G,ab) # call output(G,ab) function
      print('\nInitial grid layout: \n',R[3])
      print('\nEnergy output (kWh) of each turbine (including negative values):\n',np.round(R[0],0)) # print the array with intger values
      print('\nTotal power output of inital array is\n',R[2],'kWh.\n')
      x=R[0].flatten() # initial layout grid
      y=R[1].flatten() # power output grid
      w=R[2] # total power output

      # Optimisation model:
      m.Maximize(m.sum([y[i] for i in range(100)])) # maximisation function of total power output
      E1=m.Equation(m.sum([x[i] for i in range(100)])==T) # "Subject-to" function sets the number of turbines to user input 'T'
      E2=m.Equation(m.sum([y[i] for i in range(100)])>=w) # "subject-to" function requires total power output to increase post-optimisation

      for i in range (0,19): # set max. iterations to 20
        try:
          m.solve(disp=False) # suppress console output
          X=np.asarray(x) # convert gekko variable list into array
          X=np.reshape(X,(10,10)) # reshape above array into the dimensions of the wind farm
          print('\nOptimised grid layout: \n',X)
          print('\nEnergy output (kWh) of each turbine (including negative values):\n',np.round(X[0],0),'kWh.')
          print('\nTotal power output: ',__,'kWh.')
          print('\nIncrease in power output: ',__,'kWh.')
          break
        # Handle exceptions:
        except Exception as e:
          print(f"An error occurred: {e}")
          if i<19:
            i=i+1 # Increment 'i'
            continue # redo optimisation
          else:
            break # end optimisation attempts
    finally:
      shutil.rmtree(temp_dir)
  else:
    print('Please enter an integer value for the number of turbines bewteen 0 and 99.')

In [ ]:
!pip install gekko
from gekko import GEKKO
discoptpwr1(T=int(input('Enter an integer value for the number of turbines bewteen 0 and 99:\n')))